##Preprosesing

In [ ]:
# install modul
!pip install Sastrawi
! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 11.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import modul 
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import re
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
# create stemmer dan stopword removal
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop_factory = StopWordRemoverFactory()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#mendefinisikan variabel tanda baca dan stopword
arrTdBaca = string.punctuation 
arrSwindo = stop_factory.get_stop_words()

In [ ]:
#mendefinisikan fungsi bukannum untuk mengecek apakah string adalah angka
def bukannum(string):
    pattern = r"[^\d]+" # regex untuk mencocokkan karakter bukan angka
    return re.match(pattern, string) is not None

In [ ]:
#menghilangkan tag yang tidak diperlukan
def cleansing_text(text):
    cleaned_text = re.sub("<.*?>", "", text)
    return cleaned_text

In [ ]:
# memproses teks dengan melakukan cleansing, tokenisasi, filtering, topword removal, case folding, and stemming 
def preprocessing(text):
    # cleansing text
    text = cleansing_text(text)

    # tokenization
    tokens = word_tokenize(text)

    # filtering punctuation and stopword
    filtered_tokens = []
    for token in tokens:

        # filtering punctuation and stopword
        if (token not in arrTdBaca) and (token not in arrSwindo) and bukannum(token):
            filtered_tokens.append(token)

    # case folding
    filtered_tokens = [token.lower() for token in filtered_tokens]

    # stemming
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    # join tokens into string
    preprocessed_text = " ".join(stemmed_tokens)

    return preprocessed_text

# read csv file
df = pd.read_csv('Data_TA.csv')

# convert 'abstrak' column to string data type
df['Abstrak'] = df['Abstrak'].astype(str)

# preprocess text in the 'abstrak' column
df['Abstrak'] = df['Abstrak'].apply(preprocessing)

# save preprocessed data to new csv file
df.to_csv('preprocessed_data.csv', index=False)

## Text Processing

Word2Vec

In [ ]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [123]:
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_colwidth', 100)

# read preprocessed data
df = pd.read_csv('preprocessed_data.csv')

# handling missing value
df['Abstrak'] = df['Abstrak'].fillna('')

# label encoding
label_encoder = LabelEncoder()
df['Label_encoded'] = label_encoder.fit_transform(df['Label'])

# tokenize text
sentences = [sentence.split() for sentence in df['Abstrak']]

# split dataset into training and testing data
X_train, X_test, y_train, y_test = train_test_split(sentences, df['Label_encoded'], test_size=0.2)

# Train the word2vec model
w2v_model = gensim.models.Word2Vec(X_train,vector_size=100,window=5, min_count=2)

# save trained model
w2v_model.save('word2vec_cbow.model')

#w2v_model.wv.index_to_key
print("Shape of X_train:", np.shape(X_train))
print("Shape of X_test:", np.shape(X_test))

Shape of X_train: (682,)
Shape of X_test: (171,)


/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


CountVectorizer

In [124]:
from sklearn.feature_extraction.text import CountVectorizer

for i in range(len(X_train)):
    X_train[i] = str(X_train[i])

for i in range(len(X_test)):
    X_test[i] = str(X_test[i])

# create a CountVectorizer object
vectorizer = CountVectorizer()

# fit and transform the training data
X_train_vect = vectorizer.fit_transform(X_train)

# transform the testing data
X_test_vect = vectorizer.transform(X_test)


Naive Bayes

In [125]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import numpy as np

# train Naive Bayes model
nb = MultinomialNB()
nb.fit(X_train_vect, y_train)

# predict labels for testing data
y_pred = nb.predict(X_test_vect)

# evaluate model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# evaluate model
#print(classification_report(y_test, y_pred))


Accuracy: 0.8245614035087719


KNN

In [126]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import numpy as np

# train KNN model
knn = KNeighborsClassifier()
knn.fit(X_train_vect, y_train)

# predict labels for testing data
y_pred = knn.predict(X_test_vect)

# evaluate model
#print(classification_report(y_test, y_pred))

# evaluate model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.4152046783625731


TF IDF

In [127]:
X = df['Abstrak']
y = df['Label']
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
countvectorizer = CountVectorizer()
tfidfvectorizer = TfidfVectorizer()

count_wm = countvectorizer.fit_transform(X)
tfidf_wm = tfidfvectorizer.fit_transform(X)

In [128]:
tfidf_wm.shape

(853, 6781)

PCA

In [129]:
# Impor library yang dibutuhkan
from sklearn.decomposition import PCA

# Inisialisasi objek PCA dengan 3 komponen
pca = PCA(n_components=3)

for i in range(len(X)):
    X[i] = str(X[i])

# Melakukan fit transform pada data
X_pca = pca.fit_transform(tfidf_wm.toarray())


<ipython-input-129-17572f382588>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i] = str(X[i])


In [130]:
X_pca.shape

(853, 3)

In [134]:
from sklearn.model_selection import train_test_split
# split data into training and testing sets
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.2, random_state=1)

In [146]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import numpy as np

# train Naive Bayes model
nb = MultinomialNB()
nb.fit(X_train_pca, y_train_pca)

# predict labels for testing data
y_pred = nb.predict(X_test)

# evaluate model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

ValueError: ignored

In [145]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import numpy as np

# train KNN model
knn = KNeighborsClassifier()
knn.fit(X_train_pca, y_train_pca)

# predict labels for testing data
y_pred = knn.predict(X_test_pca)

# evaluate model
#print(classification_report(y_test, y_pred))

# evaluate model
accuracy = accuracy_score(y_test_pca, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7283950617283951
